<a href="https://colab.research.google.com/github/mihatronych/back_crowdsource/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Memes_classification_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/cozek/memotion2020-code
!pip install --upgrade efficientnet-pytorch   
!git clone https://github.com/huggingface/transformers
!pip install transformers/

Cloning into 'memotion2020-code'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 7), reused 27 (delta 4), pack-reused 0
Unpacking objects: 100% (35/35), done.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=a82b8933369392426d6b0374697ea7d6a0c8c242dbe491f33d452dde4ceb5d5b
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
Cloning into 'transformers'...
remote: Enumerating objects: 91805, done.
remote: Total 91805 (delta 0), reused 0 (delta 0), pack-reused 91805
Receiving objects: 100% (91805/91805), 75.95 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (66287/66287), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary 

In [2]:
!pwd

/content


In [3]:
cd memotion2020-code/

/content/memotion2020-code


In [4]:
cd src

/content/memotion2020-code/src


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(42)
import re
import os
import sys
sys.path.append('/kaggle/working/memotion2020-code/src/')
import pickle
import collections

from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings  
warnings.filterwarnings('ignore')

In [6]:
from typing import Callable
from tqdm import notebook
import importlib
import nltk
import datetime
import time
from argparse import Namespace
import random

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
torch.__version__

'1.10.0+cu111'

In [8]:
from efficientnet_pytorch import EfficientNet

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [10]:
!pip install keras-rectified-adam

  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.19.0-py3-none-any.whl size=9017 sha256=50ba0b551507ea78ee68bc3eb349b7fa0336033e8d3471d25884c099ae873693
  Stored in directory: /root/.cache/pip/wheels/52/1d/66/374c85b327ea412af5301b4811088408d9764314d6cfb8f127
Successfully built keras-rectified-adam


In [11]:
pip install keras-lookahead

  Created wheel for keras-lookahead: filename=keras_lookahead-0.9.0-py3-none-any.whl size=6401 sha256=5a06d799ef229041c443dfa6be3dde8795e8de4f626c28e14d95c42fdd522845
  Stored in directory: /root/.cache/pip/wheels/aa/be/c8/4dbfb810bc151428c3f631b861272250454fbe2cde6e1d089a
Successfully built keras-lookahead


In [12]:
import math
import torch
import itertools as it
from torch.optim import Optimizer
from collections import defaultdict

class Lookahead(Optimizer):
    '''
    PyTorch implementation of the lookahead wrapper.
    Lookahead Optimizer: https://arxiv.org/abs/1907.08610
    '''
    def __init__(self, optimizer,alpha=0.5, k=6,pullback_momentum="none"):
        '''
        :param optimizer:inner optimizer
        :param k (int): number of lookahead steps
        :param alpha(float): linear interpolation factor. 1.0 recovers the inner optimizer.
        :param pullback_momentum (str): change to inner optimizer momentum on interpolation update
        '''
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        self.optimizer = optimizer
        self.param_groups = self.optimizer.param_groups
        self.alpha = alpha
        self.k = k
        self.step_counter = 0
        assert pullback_momentum in ["reset", "pullback", "none"]
        self.pullback_momentum = pullback_momentum
        self.state = defaultdict(dict)

        # Cache the current optimizer parameters
        for group in self.optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                param_state['cached_params'] = torch.zeros_like(p.data)
                param_state['cached_params'].copy_(p.data)

    def __getstate__(self):
        return {
            'state': self.state,
            'optimizer': self.optimizer,
            'alpha': self.alpha,
            'step_counter': self.step_counter,
            'k':self.k,
            'pullback_momentum': self.pullback_momentum
        }

    def zero_grad(self):
        self.optimizer.zero_grad()

    def state_dict(self):
        return self.optimizer.state_dict()

    def load_state_dict(self, state_dict):
        self.optimizer.load_state_dict(state_dict)

    def _backup_and_load_cache(self):
        """Useful for performing evaluation on the slow weights (which typically generalize better)
        """
        for group in self.optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                param_state['backup_params'] = torch.zeros_like(p.data)
                param_state['backup_params'].copy_(p.data)
                p.data.copy_(param_state['cached_params'])

    def _clear_and_load_backup(self):
        for group in self.optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                p.data.copy_(param_state['backup_params'])
                del param_state['backup_params']

    def step(self, closure=None):
        """Performs a single Lookahead optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = self.optimizer.step(closure)
        self.step_counter += 1

        if self.step_counter >= self.k:
            self.step_counter = 0
            # Lookahead and cache the current optimizer parameters
            for group in self.optimizer.param_groups:
                for p in group['params']:
                    param_state = self.state[p]
                    p.data.mul_(self.alpha).add_(1.0 - self.alpha, param_state['cached_params'])  # crucial line
                    param_state['cached_params'].copy_(p.data)
                    if self.pullback_momentum == "pullback":
                        internal_momentum = self.optimizer.state[p]["momentum_buffer"]
                        self.optimizer.state[p]["momentum_buffer"] = internal_momentum.mul_(self.alpha).add_(
                            1.0 - self.alpha, param_state["cached_mom"])
                        param_state["cached_mom"] = self.optimizer.state[p]["momentum_buffer"]
                    elif self.pullback_momentum == "reset":
                        self.optimizer.state[p]["momentum_buffer"] = torch.zeros_like(p.data)

        return loss


In [13]:
# ranger combo
from radam.radam import RAdam

In [14]:
from transformers import RobertaTokenizer, RobertaModel

In [15]:
import memotion_utils.general as general_utils 
import memotion_utils.transformer.data as transformer_data_utils
import memotion_utils.transformer.general as transformer_general_utils

general_utils.set_seed_everywhere()

In [16]:
args = Namespace(
        device = 'cuda' if torch.cuda.is_available() else 'cpu',
        n_workers = 2,
        date = datetime.datetime.now().strftime("%a_%d_%b_%Y/"),
        learning_rate = 0.0001,
        num_epochs = 20,
    )

Reading files

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!ls

__init__.py  lookahead	memotion_utils	radam


In [19]:
#IMAGES_DIR = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images'
IMAGES_DIR = '/content/drive/MyDrive/memotion_dataset_7k/images'
print(os.DirEntry)
image_filenames = os.listdir(IMAGES_DIR)
file_extentions = [filename.split('.')[-1] for filename in image_filenames]

images_paths = [os.path.join(IMAGES_DIR,filename) for filename in image_filenames]

REF_FILE = '/content/drive/MyDrive/memotion_dataset_7k/reference_df_pickle'
LABELS_FILE = '/content/drive/MyDrive/memotion_dataset_7k/labels_pd_pickle'

with open(REF_FILE, 'rb') as handle:
    reference_df_ = pickle.load(handle)

with open(LABELS_FILE, 'rb') as handle:
    labels_pd_ = pickle.load(handle)
    


<class 'posix.DirEntry'>


In [20]:
image_formats = collections.Counter(file_extentions)
print(f'Num Images: {len(images_paths)}')

print('Image formats found: ', image_formats)
image_formats_df = pd.DataFrame.from_dict(image_formats, orient='index').reset_index()
image_formats_df

Num Images: 6992
Image formats found:  Counter({'jpg': 4949, 'png': 1675, 'jpeg': 345, 'JPG': 16, 'PNG': 4, 'bmp': 2, 'jpe': 1})


,index,0
0,jpg,4949
1,png,1675
2,jpeg,345
3,JPG,16
4,PNG,4
5,jpe,1
6,bmp,2


In [21]:
def get_train_val_split(train_frac, df, id_col):
    """
    Splits dataframe into train and val keeping percentage of
    labels same in both splits.
    Args:
        train_frac: Fraction of samples to use for train
        df: pd.DataFrame to split
        id_col: Column that uniquely identifies every row.
    Returns:
        split_df
    """
    val_frac = 1 - train_frac
    assert val_frac + train_frac == 1
    labels = set(df.label)
    split_df = None
    df = df.sample(frac=1) #shuffle df

    for lbl in notebook.tqdm(labels, total = len(labels)):
        lbl_df = df[df.label == lbl].copy()
        temp_df_train = lbl_df.sample(frac=train_frac).copy()
        temp_df_val = lbl_df[~lbl_df[id_col].isin(temp_df_train[id_col])].copy()
        temp_df_train['split'] = 'train'
        temp_df_val['split'] = 'val'
        if not isinstance(split_df,pd.DataFrame):
            split_df = temp_df_train.copy()
            split_df = pd.concat([split_df, temp_df_val])
        else:
            split_df = pd.concat([split_df, temp_df_train, temp_df_val])
    
    assert len(split_df) == len(df)
    return split_df

Классификатор!

In [22]:
print( f' Humor labels: {set(labels_pd_["humour"])}')
print( f' Sarcasm labels: {set(labels_pd_["sarcasm"])}')
print( f' Offensive labels: {set(labels_pd_["offensive"])}')
print( f' Motivational labels: {set(labels_pd_["motivational"])}')


humour_labels_dict = {'funny':1, 'hilarious':1, 'not_funny':0, 'very_funny':1}
sarcasm_labels_dict = {'general':1, 'twisted_meaning':1, 'not_sarcastic':0, 'very_twisted':1}
motivational_labels_dict = { 'motivational':1, 'not_motivational':0 }
offensive_labels_dict = { 'hateful_offensive':1, 'slight':1, 'not_offensive':0, 'very_offensive':1}

task_b_labels_df = labels_pd_.copy()

task_b_labels_df['humour'] = labels_pd_['humour'].map(humour_labels_dict)
task_b_labels_df['sarcasm'] = labels_pd_['sarcasm'].map(sarcasm_labels_dict)
task_b_labels_df['offensive'] = labels_pd_['offensive'].map(offensive_labels_dict)
task_b_labels_df['motivational'] = labels_pd_['motivational'].map(motivational_labels_dict)


 Humor labels: {'very_funny', 'not_funny', 'funny', 'hilarious'}
 Sarcasm labels: {'not_sarcastic', 'general', 'twisted_meaning', 'very_twisted'}
 Offensive labels: {'hateful_offensive', 'slight', 'not_offensive', 'very_offensive'}
 Motivational labels: {'not_motivational', 'motivational'}


In [23]:
print(task_b_labels_df.humour.value_counts(),'\n')
print(task_b_labels_df.sarcasm.value_counts(),'\n')
print(task_b_labels_df.offensive.value_counts(),'\n')
print(task_b_labels_df.motivational.value_counts(),'\n')

print('Total:\n',
     pd.concat(
        [
            task_b_labels_df['humour'],
            task_b_labels_df['sarcasm'],
            task_b_labels_df['offensive'],
            task_b_labels_df['motivational'],
        ],
        ignore_index= True,
        axis = 0,
    ).value_counts()      
)

1    5341
0    1651
Name: humour, dtype: int64 

1    5448
0    1544
Name: sarcasm, dtype: int64 

1    4279
0    2713
Name: offensive, dtype: int64 

0    4525
1    2467
Name: motivational, dtype: int64 

Total:
 1    17535
0    10433
dtype: int64


In [24]:
class MemotionImageDataset(Dataset):
    def __init__(self, data_df: pd.DataFrame , images_dir:str = None):
        """
        Args:
            data_df (pandas.DataFrame): df containing the labels and text
            tokenizer (tokenizer module for the transformer)
        """

        self.images_dir = IMAGES_DIR
        
        self.data_df = data_df
        
        self.train_df = self.data_df[self.data_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.data_df[self.data_df.split == 'val']
        self.val_size = len(self.val_df)

        
        self._lookup_dict = {
            'train': (self.train_df, self.train_size),
            'val': (self.val_df, self.val_size),
        }

        self.set_split('train')

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        img_name = row.image_name

        normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        if self._target_split == 'train':
            tfms = transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
        elif self._target_split =='val':
            tfms = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])
            
        img_path = os.path.join(self.images_dir,img_name) 
        img = Image.open(img_path)
        img = img.convert('RGB')
        preproc_img = tfms(img)

        label = row.label
        
        return {
            'x_data': preproc_img,
            'x_index': index,
            'y_target': label
        }

    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

In [25]:
def generate_image_batches(dataset, batch_size, shuffle=True,
                     drop_last=False, device="cpu", pinned_memory = False, n_workers = 0): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            pin_memory= pinned_memory,
                            num_workers = n_workers,
                            )
    
    for data_dict in dataloader:
        out_data_dict = {}
        out_data_dict['x_data'] = data_dict['x_data'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_index'] = data_dict['x_index']
        out_data_dict['y_target'] = data_dict['y_target'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        yield out_data_dict

In [45]:
humor_df = task_b_labels_df[['image_name','offensive']].copy()
humor_df.rename(columns = {'offensive':'label'}, inplace=True)
humor_df = get_train_val_split(0.90, humor_df,'image_name')

  0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
humor_df[humor_df.split == 'train'].label.value_counts()

1    3851
0    2442
Name: label, dtype: int64

In [47]:
humor_df[humor_df.split == 'val'].label.value_counts()

1    428
0    271
Name: label, dtype: int64

In [49]:
model = EfficientNet.from_pretrained(
    'efficientnet-b4',
    num_classes = len(set(humor_df.label)),
 )
model.to(args.device)

Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [50]:
dataset = MemotionImageDataset(humor_df)

In [51]:
loss_func = nn.CrossEntropyLoss()
print(f'Using LR:{args.learning_rate}')
base_optimizer = RAdam(model.parameters(), lr = args.learning_rate)
optimizer = Lookahead(optimizer = base_optimizer, k = 5, alpha=0.5 )
early_stopping = transformer_general_utils.EarlyStopping(patience=2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

Using LR:0.0001


In [52]:
args.batch_size = 32

In [53]:
train_state = general_utils.make_train_state()
train_state.keys()

dict_keys(['train_preds', 'train_indexes', 'train_targets', 'train_accuracies', 'train_f1s', 'train_losses', 'val_preds', 'val_indexes', 'val_targets', 'val_accuracies', 'val_f1s', 'val_losses', 'test_preds', 'test_indexes', 'test_targets', 'test_accuracies', 'test_f1s', 'test_losses', 'batch_preds', 'batch_targets', 'batch_indexes', 'epoch_index'])

In [54]:
epoch_bar = notebook.tqdm(
    desc = 'training_routine',
    total = args.num_epochs,
    position=0,
    leave = True,
)
dataset.set_split('train')
train_bar = notebook.tqdm(
    desc = 'split=train ',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)
dataset.set_split('val')
eval_bar = notebook.tqdm(
    desc = 'split=eval',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)
for epoch_index in range(args.num_epochs):
    train_state['epoch_in'] = epoch_index

    dataset.set_split('train')

    batch_generator = generate_image_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = False, n_workers = 2, 
    )

    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    model.train()

    train_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )

    for batch_index, batch_dict in enumerate(batch_generator):
        optimizer.zero_grad()
        
        y_pred = model(batch_dict['x_data'])
        
        y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
        
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss.backward()
        optimizer.step()
                             
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)
                             
        y_pred = y_pred.detach().cpu()
        batch_dict['y_target'] = batch_dict['y_target'].cpu()
        
        acc_t = transformer_general_utils \
            .compute_accuracy(y_pred, batch_dict['y_target'])
        
        f1_t = transformer_general_utils \
            .compute_macro_f1(y_pred, batch_dict['y_target'])

        train_state['batch_preds'].append(y_pred)
        train_state['batch_targets'].append(batch_dict['y_target'])
        train_state['batch_indexes'].append(batch_dict['x_index'])

        running_acc += (acc_t - running_acc) / (batch_index + 1)
        running_f1 += (f1_t - running_f1) / (batch_index + 1)

        train_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                             epoch=epoch_index)

        train_bar.update()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_state['train_accuracies'].append(running_acc)
    train_state['train_losses'].append(running_loss)
    
    train_state['train_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )
    train_state['train_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['train_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    train_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['train_preds'][-1],
                                  train_state['train_targets'][-1],
                                 )
                                 
    train_state['train_f1s'].append(train_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    
    dataset.set_split('val')
    batch_generator = generate_image_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = False, n_workers = 2, 
    )
    eval_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    
    model.eval()
    with torch.no_grad():
        optimizer._backup_and_load_cache()
        for batch_index, batch_dict in enumerate(batch_generator):
            y_pred = model(batch_dict['x_data'])
            y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            y_pred = y_pred.detach().cpu()
            batch_dict['y_target'] = batch_dict['y_target'].cpu()
            
            acc_t = transformer_general_utils\
                .compute_accuracy(y_pred, batch_dict['y_target'])
            f1_t = transformer_general_utils \
                .compute_macro_f1(y_pred, batch_dict['y_target'])

            train_state['batch_preds'].append(y_pred.cpu())
            train_state['batch_targets'].append(batch_dict['y_target'].cpu())
            train_state['batch_indexes'].append(batch_dict['x_index'].cpu())

            running_acc += (acc_t - running_acc) / (batch_index + 1)
            running_f1 += (f1_t - running_f1) / (batch_index + 1)
            

            eval_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                                 epoch=epoch_index)
            eval_bar.update()
            
    train_state['val_accuracies'].append(running_acc)
    train_state['val_losses'].append(running_loss)
    
        
    train_state['val_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )

    train_state['val_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['val_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    val_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['val_preds'][-1],
                                  train_state['val_targets'][-1],
                                 )
                                 
    train_state['val_f1s'].append(val_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    scheduler.step(val_f1)
    early_stopping(val_f1, model)
    optimizer._clear_and_load_backup()
    epoch_bar.set_postfix( best_f1 = early_stopping.best_score, current = val_f1)
    epoch_bar.update()    
    
    if early_stopping.early_stop:
        print("Early stopping")
        break
        

training_routine:   0%|          | 0/20 [00:00<?, ?it/s]

split=train :   0%|          | 0/196 [00:00<?, ?it/s]

split=eval:   0%|          | 0/21 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 2
EarlyStopping counter: 2 out of 2
Early stopping


In [55]:
import pickle

In [57]:
with open('/content/drive/MyDrive/memotion_dataset_7k/data.pickle', 'wb') as f:
  pickle.dump(model, f)

In [116]:
import json
from PIL import Image
import torch
from torchvision import transforms

from efficientnet_pytorch import EfficientNet

# Preprocess image

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
tfms = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])        
img = Image.open('/content/drive/MyDrive/memotion_dataset_7k/images/image_105.png')
img = img.convert('RGB')
preproc_img = tfms(img).unsqueeze(0)

print(preproc_img.shape) # torch.Size([1, 3, 224, 224])

# Load ImageNet class names


# Classify
model.eval()
with torch.no_grad():
    outputs = model(preproc_img.cuda())

# Print predictions
print('-----')
for idx in torch.topk(outputs, k=1).indices.squeeze(0).tolist():
    prob = torch.softmax(outputs, dim=1)[0, idx].item()
    print('({p:.2f}%)'.format( p=prob*100))

torch.Size([1, 3, 224, 224])
-----
(54.85%)
